In [6]:
# Imports
import requests
import yaml

from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
# Reading config
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)
    address = config["Providers"]["DEEPSEEKLOCAL"]

In [53]:
# Defining API calls
test = "/v1/models"
completions = "/v1/chat/completions"

In [54]:
# Testing
x = requests.get(address + test)
x.content

b'{\n  "data": [\n    {\n      "id": "deepseek-r1-distill-qwen-7b",\n      "object": "model",\n      "owned_by": "organization_owner"\n    },\n    {\n      "id": "text-embedding-nomic-embed-text-v1.5",\n      "object": "model",\n      "owned_by": "organization_owner"\n    }\n  ],\n  "object": "list"\n}'

In [55]:
# DeepSeek R1 Qwen model
class DEEPSEEKR1QWEN:
    def __init__(self) -> None:
        with open("/Users/fitter0happier/Desktop/Coding/NLP/21Mind/config.yaml", "r") as file:
            config = yaml.safe_load(file)

        self.base_url = config["Providers"]["DEEPSEEKLOCAL"]
        self.model = config["IDs"]["DEEPSEEKR1QWEN"]
        self.completions = "/v1/chat/completions"

    def response(self, mes):
        headers = {
            "Content-Type": "application/json"
        }
        data = {
            "model": self.model,
            "messages": mes,
            "temperature": 0.6,
            "max_tokens": -1,
            "stream": False
        }

        response = requests.post(self.base_url+self.completions, headers=headers, json=data).json()
        return response['choices'][0]['message']['content']

In [11]:
message=[
    {
      "role": "user",
      "content": """
        You are playing BlackJack, the dealer has a 7 of Spades, 
        you have a Queen of Hearts and a Two of Diamonds, 
        availabe actions are Hit and Stand. 
        What is your action? 
      """
    }
]

In [8]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B", device_map="auto")  # Automatically uses GPU if available

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
input_ids = tokenizer.apply_chat_template(
    message,
    add_generation_prompt=True,  # Adds the assistant's start token to trigger generation
    return_tensors="pt"
).to(model.device)

In [14]:
outputs = model.generate(
    input_ids,
    max_new_tokens=10000,  # Controls the maximum length of the generated text
    temperature=0.6,     # Lower temperature for more deterministic outputs
    top_p=0.9,           # Nucleus sampling: chooses from the top p probability mass
    eos_token_id=tokenizer.eos_token_id  # Stop generation at the EOS token
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [15]:
generated_ids = outputs[:, input_ids.shape[1]:]
response = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(response)

Okay, so I'm trying to figure out whether to hit or stand in this BlackJack situation. Let me break it down step by step.

First, the dealer has a 7 of Spades. That means the dealer's up card is a 7. In Blackjack, the dealer's up card is crucial because it gives the player an idea of what the dealer might be holding. The dealer will hit until they reach at least 17, so knowing their current hand can help decide when to stand.

Now, the player has a Queen of Hearts and a Two of Diamonds. Let me add those up. The Queen is 12, and the Two is 2, so that's 14 in total. In Blackjack, 14 is a soft 14 because if you have an Ace, it can count as 11. Wait, no, the player only has a Queen and a Two, so it's just 14. That's a hard 14 because there's no Ace to make it soft.

The dealer's 7 is showing, so the dealer's current hand is 7 plus whatever they're holding. The dealer will hit until they reach 17 or more. So, if the dealer's total is below 17, they'll hit. If it's 17 or higher, they'll stan